## Goals

So, let us continue the sprucing and finally prepare the dataset for modeling (initially without concern for the other variables).

In [225]:
import pandas as pd
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import regex as re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [3]:
df = pd.read_csv('../data/cleaned_02_i.csv')
print(df.shape)
df.head()

(39565, 56)


,platform,text,hate_speech_score,target_race_asian,target_race_black,target_race_latinx,target_race_middle_eastern,target_race_native_american,target_race_pacific_islander,target_race_white,...,target_age_other,target_age,target_disability_physical,target_disability_cognitive,target_disability_neurological,target_disability_visually_impaired,target_disability_hearing_impaired,target_disability_unspecific,target_disability_other,target_disability
0,0,White people are trash.,0.46,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,"You've caught the big gay, it's known to be ai...",0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,"I'm not saying you said that, I'm just saying ...",-1.29,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,Donald Trump. Yeet myself off a building onto ...,-0.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,Fabrice Fabrice is ostensibly black or black/l...,-2.84,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


https://huggingface.co/datasets/ucberkeley-dlab/measuring-hate-speech <br>
hate_speech_score - continuous hate speech measure, where higher = more hateful and lower = less hateful. > 0.5 is approximately hate speech, < -1 is counter or supportive speech, and -1 to +0.5 is neutral or ambiguous.

In [4]:
#First up is to rename the hate_speech_score to a more generic label.
df['hate_speech_score'].describe() #Again, recall the words copied above fom their website. So, for my purposes I'd be interested
#in potentially intervening in people's behaviour. Likely that would start the second category at least at .5;
#and we'd still need to figure out the gauge for the IMMEDIATE category.
#Part of what might be difficult is the subjectivity in joking and the like.More and more I'm thinking to perhaps go back and
#grab those other columns I ignored, such as 'genocide'..... However, for now when we're just doing language we'll work as is
#(anyways, i this case I'd rewrite/append to the quoted current source).



count    39565.000000
mean        -0.943911
std          2.073087
min         -8.340000
25%         -2.330000
50%         -0.700000
75%          0.570000
max          6.300000
Name: hate_speech_score, dtype: float64

In [134]:
i = -1

In [160]:
i += 1
print(df.iloc[i, 2], i)
df.iloc[i, 1]

0.8 24


"OKAY, I'LL LEAVE YOU FUCK YOURSELF LIKE A FUCKING DOG BARKING, BECAUSE I DON'T GIVE A FUCK TO A FUCKING BITCH LIKE YOU! I HOPE YOUR DICK-FACE FALLS ON A PIECE OF CRAP, HAVE A CRAPPY DAY, JACKARSE"

### Introducing the first hyperparameter - 'benign', 'concerning', 'malevonent'
Corresponding they'll be known as 0, 1, and 2 respectively. However, please note - likely I may tweak these values later. Recall that initialy these 'predictions' are categorical and I'm somewhat artificially assigning such classes to them for my purposes.
A further difficulty will be the trend I'v already noticed of 'counter/supportive speach' often being quite graphc as well. Besides fom a NLP point of view making things more difficult, definitionally it would seem to me crass and crude to refer to things in such a boorish manner. Ie perhaps I am biased, however I would hope supportive speach is not as filthy as hate speach, with the only difference being the direction it is aimed at.

On the topic of other potential parameters: Pay attention to how I'll also, later on in this section, clean the various texts. Perhaps it'll be worthwhile to experiment with that.

In [130]:
lower_limit, upper_limit = -1, 2 #1.41 makes the two big groups approx. equal

In [131]:
df['hate_rank'] = [2 if i > upper_limit
                  else 1 if i > lower_limit else 0
                  for i in df['hate_speech_score']]

In [132]:
df['hate_rank'].value_counts(normalize=True) #Hmm, eh whatever - we'll keep it at this propotion for now.
#To write it out in a comment here,in case I change it later: .44,.5, .06 resp.

hate_rank
1    0.503627
0    0.443650
2    0.052723
Name: proportion, dtype: float64

In [201]:
#Hmm, something else I noticed while going through posts is the useage of capitals. Perhaps we should go ahead and make a
#contains lots of capitals column:
samp = "OKAY, I'LL LEAVE YOU FUCK YOURSELF LIKE A FUCKING DOG BARKING, BECAUSE I DON'T GIVE A FUCK TO A FUCKING BITCH LIKE YOU! I HOPE YOUR DICK-FACE FALLS ON A PIECE OF CRAP, HAVE A CRAPPY DAY, JACKARSE"

In [217]:
def capital_checker(sentance, thresh=.2):
    count, per_count = 0, 0
    for i in range(len(sentance)):
        if sentance[i].isupper() == True:
            count += 1
        if sentance[i] == '.':
            per_count += 1
    if (count - per_count)/len(sentance) >= thresh and count > per_count: #Ie the sentance has an overwhelming capital presence...
        verdict = 1
    else:
        verdict = 0
    return verdict

In [218]:
df['capital_sentance'] = df['text'].apply(capital_checker)

In [224]:
df['capital_sentance'].value_counts(normalize=True)

capital_sentance
0    0.966056
1    0.033944
Name: proportion, dtype: float64

In [226]:
#Great, now we can move foreward with NLPing, finally.Pragmatically it might be worthwhile to playaround with not lowercasing
#the words, but for now we'll go with the 'default' approach of doing so, in an attempt to reduce the upcoming flood of featues.

#While I'm doing all this, I might as well make a word counter as well. That might be useful later.

#Hmm, let's start by breaking apart the comments, trying to get an aveage word count...
def document_words(raw_document):

    #Remove some various html hiccups that may have crept in.
    document_text = BeautifulSoup(raw_document).get_text()
    
    #Further purge it via regex to keep only characters per Regex. For sure in this case I'm keeping numbers
    letters_only = re.sub("[^a-zA-Z0-9]", " ", document_text)
    #Convert to lower case, split into individual words.
    
    words = letters_only.split()
    words = [w.lower() for w in words]
 
    p_stemmer = PorterStemmer()
    stemmed_words = [p_stemmer.stem(i, to_lowercase=False) for i in words]
    
    #And finally, let us merge back everything to a new document.
    return(" ".join(stemmed_words))

In [227]:
 df['cleaned_text'] = df['text'].apply(document_words)

C:\Users\bwolf\AppData\Local\Temp\ipykernel_14960\3250074403.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  document_text = BeautifulSoup(raw_document).get_text()


In [228]:
df['wc_cleaned_text'] =  df['cleaned_text'].str.split().str.len()

In [229]:
df['wc_cleaned_text'].describe()

count    39565.000000
mean        25.779932
std         20.312349
min          1.000000
25%         11.000000
50%         20.000000
75%         35.000000
max        128.000000
Name: wc_cleaned_text, dtype: float64

In [230]:
df.groupby('hate_rank')['wc_cleaned_text'].describe()#Hmm, pehaps there is a trend here afterall - 0 is longer on
#all fronts. Ie perhaps when one uses words more, as opposed to being rash and simply labeling things in a smaller word insult
#that can be easily digested, that is a sign of a more nuanced person. However,such differences are negligible so perhaps
#it is not all that significant. Regardless, we continue.

,count,mean,std,min,25%,50%,75%,max
hate_rank,,,,,,,,
0,17553.0,28.759528,21.745569,1.0,13.0,23.0,40.00,128.0
1,19926.0,23.293938,18.695904,2.0,10.0,18.0,32.00,124.0
2,2086.0,24.454458,19.301883,2.0,10.0,19.0,32.75,117.0


In [232]:
#Hmm, yeah, I guess we'll export these
df.to_csv('../data/cleaned_02_ii.csv', index=False)